## 1.4 Runnables

<div style="text-align: right"> Initial issue : 2025.04.26 </div>
<div style="text-align: right"> last update : 2025.04.26 </div>

`Runnable이란`  
- Langchain에서 워크플로우나 체인 구성을 위해 사용하는 프로토콜(인터페이스)
- 모델 관련 컴포넌트(프롬프트, LLM, 출력파서, 리트리버 등)를 표준화된 방식으로 조합하고 실행하도록 하는 역할

Langchain에서는 다양한 `Runnable` 구현체를 제공함   
- 병렬 처리: Parrallel
- 데이터 전달
    - `RunnablePassthrough`: 입력을 변경하지 않거나 추가 키를 더하여 전달할 
    - `RunnablePassthrough()` 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달
    - `RunnablePassthrough.assign(...)` : 입력을 받아 assign 함수에 전달된 추가 인수를 추가
- 사용자 정의 함수 매핑: RunnableLambda

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils import langsmith
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Parallel: 병렬성  
- `langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 병렬 실행가능

In [4]:
from langchain_core.runnables import RunnableParallel

In [5]:
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디인가요?")
    | model
    | StrOutputParser()
)
chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마인가요?")
    | model
    | StrOutputParser()
)
combined_chain = RunnableParallel(capital = chain1, area = chain2)

In [6]:
combined_chain.invoke({"country": "한국"})

{'capital': '한국의 수도는 서울입니다.',
 'area': '한국의 면적은 약 100,210 평방킬로미터입니다. 이는 한반도의 남쪽에 위치한 대한민국의 면적을 기준으로 하며, 북한을 포함한 한반도의 전체 면적은 약 220,000 평방킬로미터입니다.'}

병렬 처리

In [7]:
combined_chain.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울입니다.',
  'area': '대한민국의 면적은 약 100,210 평방킬로미터(㎢)입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과의 경계를 포함한 전체 면적입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '미국의 면적은 약 9,830,000 평방킬로미터(3,796,000 평방마일)입니다. 이는 미국이 세계에서 세 번째로 큰 나라임을 의미합니다. 러시아와 캐나다에 이어 면적이 가장 큰 국가입니다.'}]

### RunnablePassthrough
`RunnablePassthrough` 는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메소드를 사용하여 별도 실행 가능

In [8]:
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)
chain = prompt | llm

invoke를 실행할 때는 입력 데이터의 타입이 딕셔너리여야 한다.

In [9]:
chain.invoke({"num": 10})

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQtMwMd2Wmo8C6K8v8V1XWktUdsMS', 'finish_reason': 'stop', 'logprobs': None}, id='run-1089e666-47a5-4b19-8a38-8b7cf6d6be2f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 4, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

하지만 프롬프트에 1개의 변수만 있다면 값만 전달하는 것도 가능하다.

In [10]:
chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQtMwsnaQOUXtpm00ZLkh2bYKpmmZ', 'finish_reason': 'stop', 'logprobs': None}, id='run-30f36e6e-b558-40e8-ade7-b3f4e1578adf-0', usage_metadata={'input_tokens': 16, 'output_tokens': 4, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

모든 runnable 객체는 invoke 메서드 사용 가능

In [11]:
from langchain_core.runnables import RunnablePassthrough
RunnablePassthrough().invoke(10)

10

In [12]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
runnable_chain.invoke(10) # dict가 RunnablePassthrough 로 변경됨

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQtMxJGdnsln78TDCTjDo65rYt4Wg', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2e78814-07b1-4ee9-a3dc-0356c2940fd6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 4, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
runnable_chain.invoke({"num": 10})

AIMessage(content='100입니다. 10 * 10 = 100.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 21, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQtMy8D2fTTDwCxsympCOknifYuZz', 'finish_reason': 'stop', 'logprobs': None}, id='run-afc95d8b-96ab-4e65-abb9-8db15f040ef0-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

`RunnablePassthrough.assign(...)` 방식으로 호출되면, 입력을 받아 assgin 함수에 전달된 함수를 추가







In [14]:
RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3).invoke({"num": 1})

{'num': 1, 'new_num': 3}

In [15]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

`RunnableParallel` 응용

In [16]:
# Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

### RunnableLambda: 사용자 정의 함수 매핑

먼저 간단한 함수를 정의

In [17]:
from datetime import datetime

def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")

get_today(None)

'Apr-27'

- 여기서 주의할 점은 사용하지는 않지만 매개변수를 무조건 지정해야한다는 점

In [18]:
from langchain_core.runnables import RunnableLambda

prompt = PromptTemplate.from_template(
    "{today}가 생일인 {n} 명을 나열하고, 생년월일을 표기해주세요."
)
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
print(chain.invoke({"n": 3}))

다음은 4월 27일에 생일인 유명인 몇 명입니다:

1. **사무엘 고든** (Samuel Gordon) - 1940년 4월 27일
2. **우디 패럴** (Woody Harrelson) - 1961년 4월 27일
3. **제이슨 베이트먼** (Jason Bateman) - 1969년 4월 27일

이 외에도 4월 27일에 태어난 많은 사람들이 있습니다!


In [20]:
print(chain.invoke(3))

다음은 4월 27일에 태어난 유명인 3명입니다:

1. **이사벨 아옌데 (Isabel Allende)** - 1942년 4월 27일
2. **세라 제시카 파커 (Sarah Jessica Parker)** - 1965년 4월 27일
3. **우디 해럴슨 (Woody Harrelson)** - 1961년 4월 27일

이 외에도 4월 27일에 태어난 많은 사람들이 있습니다!


주의할 점
- 위에서 get_today 함수의 인자는 n이 된다.
- 위에서는 문제가 없었지만 `{"n": 3}`을 입력하면 get_today(3)이 들어가는 꼴이 된다.
- 따라서 `chain.invoke(3)` 형태로 사용하는 것이 바람직하다.  
- 만약 {"n": 3} 형태로 invoke 하고 싶다면 itemgetter를 사용하면 된다.

`itemgetter` 를 사용하여 특정 키를 추출하기

In [21]:
from operator import itemgetter

In [22]:
def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [24]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQtN4gBzBl1kJEoFScz7Xaa4FzRgm', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d15911d-98c3-47c1-9f7a-9249586d91f5-0', usage_metadata={'input_tokens': 22, 'output_tokens': 10, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})